In [56]:
import os

In [57]:
%pwd

'c:\\Users\\mathe\\Udemy\\Mlops_course\\section18\\data-science-project'

In [58]:
os.chdir("c:/Users/mathe/Udemy/Mlops_course/section18/data-science-project")
%pwd

'c:\\Users\\mathe\\Udemy\\Mlops_course\\section18\\data-science-project'

In [59]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path


In [60]:
from src.DataScienceProject.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.DataScienceProject.utils.common import read_yaml, create_directories

In [61]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH,params_file_path = PARAMS_FILE_PATH, schema_file_path=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation= DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path)
        return data_transformation

In [62]:
import os
from sklearn.model_selection import train_test_split
from src.DataScienceProject import logger
import pandas as pd

In [63]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_spliting(self):
        try:
            df = pd.read_csv(self.config.data_path)
            logger.info(f"Read {self.config.data_path}")
            train, test = train_test_split(df, test_size=0.3, random_state=42)

            train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
            test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

            logger.info("Train test split Completo.")
            logger.info(f"Csv de teste e treino salvo em: {self.config.root_dir}")
            logger.info(f"Train shape: {train.shape}, Test shape: {test.shape}")
        except Exception as e:
            logger.error(f"Erro no train_test_spliting: {e}")
            raise e

In [64]:
try:
    config_manager = ConfigurationManager()
    data_transformation_config = config_manager.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:  
    logger.error(f"Erro na execução do Data Transformation: {e}")
    raise e
    

[2026-02-05 15:33:52,333: INFO: common: YAML file: config\config.yaml carregado com sucesso:]
[2026-02-05 15:33:52,336: INFO: common: YAML file: params.yaml carregado com sucesso:]
[2026-02-05 15:33:52,344: INFO: common: YAML file: schema.yaml carregado com sucesso:]
[2026-02-05 15:33:52,347: INFO: common: Diretório criado em: artifacts:]
[2026-02-05 15:33:52,350: INFO: common: Diretório criado em: artifacts/data_transformation:]
[2026-02-05 15:33:52,359: INFO: 1582645047: Read artifacts/data_ingestion/winequality-red.csv:]
[2026-02-05 15:33:52,391: INFO: 1582645047: Train test split Completo.:]
[2026-02-05 15:33:52,393: INFO: 1582645047: Csv de teste e treino salvo em: artifacts/data_transformation:]
[2026-02-05 15:33:52,394: INFO: 1582645047: Train shape: (1119, 12), Test shape: (480, 12):]
